In [1]:
import pandas as pd
import numpy as np

from warehouse_modeling.induced_backorder_cost import *
from warehouse_modeling.lead_time_approximation import *
from warehouse_modeling.warehouse_optimization import *
from warehouse_modeling.warehouse_demand_modeling import *

from single_echelon_utils.inventory_level_computation import *
from single_echelon_utils.service_level_computation import *
from single_echelon_utils.dealer_optimization import *

from utils import *

## INDATA
First, read indata from a specified excel file and sheet.

In [2]:
excel_path_indata = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/9. Analytical modeling/test_indata.xlsx"
indata_sheet = "test_case_1_whNormal_NBD"
indataDF = pd.read_excel(excel_path_indata,indata_sheet)
outdataDF = indataDF.copy()
indataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev
0,1,RDC,Johannesburg,10,40,1,NaN,NaN,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.1
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.2
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.3
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,1.4
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,1.5
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,1.6
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,1.7
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,1.8
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,1.9


In [3]:
Q_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Q").to_numpy()
mu_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
sigma_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()
demand_type_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand type").to_numpy()
Q_subbatch_size = find_smallest_divisor(Q_dealer_arr)
L_wh = float(indataDF.get(indataDF["Type"]=="RDC").get("Transport time"))

In [4]:

rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(Q_dealer_arr, mu_dealer_arr, 
    sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size)

outdataDF.loc[outdataDF["Type"] == "Dealer","Q, subbatches"] = Q_dealer_arr/Q_subbatch_size

outdataDF.loc[outdataDF["Type"] == "RDC","Demand type"] = wh_dist
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand mean"] = mu_L * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Demand mean"] = mu_L * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC","Demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC", "Demand variance"] = sigma2_L * Q_subbatch_size/L_wh

outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,"Q, subbatches",Lead time demand mean,Lead time demand stdev,Demand variance
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,10.0,2.054886,NaN,100.0,20.548858,4.222556
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.100000,1.0,NaN,NaN,NaN
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.200000,1.0,NaN,NaN,NaN
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.300000,1.0,NaN,NaN,NaN
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,1.400000,1.0,NaN,NaN,NaN
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,1.500000,1.0,NaN,NaN,NaN
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,1.600000,1.0,NaN,NaN,NaN
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,1.700000,1.0,NaN,NaN,NaN
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,1.800000,1.0,NaN,NaN,NaN
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,1.900000,1.0,NaN,NaN,NaN


RDC reorder-point optimization

In [5]:
h_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Holding cost").to_numpy()
fill_rate_target_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Target item fill rate").to_numpy()
p_dealer_arr = fill_rate_target_arr*h_dealer_arr/(np.ones_like(fill_rate_target_arr)-fill_rate_target_arr)
l_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Transport time").to_numpy()
mu_wh = mu_L/L_wh * Q_subbatch_size

beta_list = []
for h,Q,p,l,my,sigma in zip(h_dealer_arr,Q_dealer_arr,p_dealer_arr,l_dealer_arr,mu_dealer_arr,sigma_dealer_arr):
    beta_list.append(induced_backorder_cost_opt(h,Q,p,l,my,sigma))

beta_arr = np.array(beta_list)

beta_rdc = weighting_backorder_cost(mu_dealer_arr,mu_wh,beta_arr)
outdataDF.loc[outdataDF["Type"] == "Dealer", "Shortage cost"] = p_dealer_arr
outdataDF.loc[outdataDF["Type"] == "RDC", "Beta"] = beta_rdc
outdataDF.loc[outdataDF["Type"] == "Dealer", "Beta"] = beta_arr
print(f"Optimal weighted induced backorder cost at the warehouse is: {beta_rdc}, betas are: {beta_arr}")



Optimal weighted induced backorder cost at the warehouse is: 0.7727785767013167, betas are: [0.30415552 0.51926117 0.55475629 0.56157126 0.77701758 0.82454386
 0.85383623 1.01616462 1.13619123 1.18028801]


In [6]:
h_rdc = float(indataDF.get(indataDF["Type"] == "RDC").get("Holding cost"))
Q_0 = int(int(indataDF.get(indataDF["Type"] == "RDC").get("Q"))/Q_subbatch_size)

R_0 = warehouse_optimization(Q_subbatch_size,Q_0,rdc_f_u_probability_array,h_rdc,beta_rdc)

outdataDF.loc[outdataDF["Type"] == "RDC", "Q, subbatches"] = Q_0
outdataDF.loc[outdataDF["Type"] == "RDC", "R, subbatches"] = R_0
outdataDF


Starting optimizing, R = 0, c = 57.963265686817934, c+1 = 50.256079454724805
Doing upwards optimizing, R = 1, c = 50.256079454724805, c+1 = 42.612039659497434
Doing upwards optimizing, R = 2, c = 42.612039659497434, c+1 = 35.1635760404851
Doing upwards optimizing, R = 3, c = 35.1635760404851, c+1 = 28.20748212705672
Doing upwards optimizing, R = 4, c = 28.20748212705672, c+1 = 22.266240970767413
Doing upwards optimizing, R = 5, c = 22.266240970767413, c+1 = 18.051740194387744
Doing upwards optimizing, R = 6, c = 18.051740194387744, c+1 = 16.283063946454774
Doing upwards optimizing, R = 7, c = 16.283063946454774, c+1 = 17.419170879724753


,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,"Q, subbatches",Lead time demand mean,Lead time demand stdev,Demand variance,Shortage cost,Beta,"R, subbatches"
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,10.0,2.054886,4.0,100.0,20.548858,4.222556,NaN,0.772779,7.0
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.100000,1.0,NaN,NaN,NaN,19.0,0.304156,NaN
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.200000,1.0,NaN,NaN,NaN,19.0,0.519261,NaN
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.300000,1.0,NaN,NaN,NaN,19.0,0.554756,NaN
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,1.400000,1.0,NaN,NaN,NaN,19.0,0.561571,NaN
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,1.500000,1.0,NaN,NaN,NaN,19.0,0.777018,NaN
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,1.600000,1.0,NaN,NaN,NaN,19.0,0.824544,NaN
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,1.700000,1.0,NaN,NaN,NaN,19.0,0.853836,NaN
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,1.800000,1.0,NaN,NaN,NaN,19.0,1.016165,NaN
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,1.900000,1.0,NaN,NaN,NaN,19.0,1.136191,NaN


In [7]:
W = waiting_time(negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array),L_wh,mu_L,Q_subbatch_size)
outdataDF.loc[outdataDF["Type"]== "Dealer", "Waiting time"] = W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Transport time").to_numpy() + W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand mean"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand mean").to_numpy()
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand stdev"] =outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand stdev").to_numpy()
outdataDF.loc[outdataDF["Type"] == "RDC", "Lead time"] = indataDF.get(indataDF["Type"] == "RDC").get("Transport time")

In [8]:
opt_dealer_list = []
L_dealer_arr = outdataDF.get(outdataDF["Type"] == "Dealer").get("Lead time")
for Q,L_est,fill_rate_target,demand_type,mu,sigma in zip(Q_dealer_arr,L_dealer_arr,fill_rate_target_arr,demand_type_arr, mu_dealer_arr,sigma_dealer_arr):
    print(demand_type, mu, sigma)
    opt_dealer_list.append(dealer_R_optimization(Q,L_est,fill_rate_target,demand_type,mu,demand_variance = math.pow(sigma,2)))

R_opt_dealer_list,fill_rate_dealer_list,exp_stock_on_hand_list = [],[],[]
for tup in opt_dealer_list:
    R_opt_dealer_list.append(tup[0])
    fill_rate_dealer_list.append(tup[2])
    exp_stock_on_hand_list.append(tup[3])

outdataDF.loc[outdataDF["Type"] == "Dealer", "R optimal"] = R_opt_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Realized item fill rate"] = fill_rate_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected stock on hand"] = exp_stock_on_hand_list

outdataDF.loc[outdataDF["Type"] == "RDC","R optimal"] = R_0*Q_subbatch_size # In units.
outdataDF.loc[outdataDF["Type"] == "RDC","Expected stock on hand"] = positive_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array)
outdataDF.loc[outdataDF["Type"] == "RDC","Expected backorders"] = negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array)


NBD 1.0 1.1
NBD 1.0 1.2
NBD 1.0 1.3
NBD 1.0 1.4
NBD 1.0 1.5
NBD 1.0 1.6
NBD 1.0 1.7
NBD 1.0 1.8
NBD 1.0 1.9
NBD 1.0 2.0


In [9]:
# List comprehension for exp backorders.
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected backorders"] = [ 
    expected_backorders_discrete(R,Q,lt_mu,exp_stock_on_hand) for 
    R,Q,lt_mu,exp_stock_on_hand in zip(R_opt_dealer_list,Q_dealer_arr,
    outdataDF.get(outdataDF["Type"] == "Dealer").get("Lead time demand mean").to_numpy(),
    exp_stock_on_hand_list) ]

In [10]:
# Adding costs
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected holding costs per time unit"] = h_dealer_arr*np.array(exp_stock_on_hand_list)
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected shortage costs per time unit"] = outdataDF.get(
    outdataDF["Type"] == "Dealer").get("Expected backorders").to_numpy()*p_dealer_arr
outdataDF.loc[outdataDF["Type"] == "Dealer", "Total expected costs"] = outdataDF.get(outdataDF["Type"] == "Dealer").get("Expected holding costs per time unit").to_numpy() + outdataDF.get(outdataDF["Type"] == "Dealer").get("Expected shortage costs per time unit").to_numpy()

outdataDF.loc[outdataDF["Type"] == "RDC", "Expected holding costs per time unit"] = h_rdc*outdataDF.get(outdataDF["Type"] == "RDC").get("Expected stock on hand").to_numpy()
outdataDF.loc[outdataDF["Type"] == "RDC", "Total expected costs"] = outdataDF.get(outdataDF["Type"] == "RDC").get("Expected holding costs per time unit").to_numpy()



In [11]:
excel_path_outdata = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/9. Analytical modeling/test_outdata.xlsx"
outdataDF.to_excel(excel_path_outdata,sheet_name = "Outdata_latest_testrun")

outdataDF


,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,...,"R, subbatches",Waiting time,Lead time,R optimal,Realized item fill rate,Expected stock on hand,Expected backorders,Expected holding costs per time unit,Expected shortage costs per time unit,Total expected costs
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,10.0,2.054886,...,7.0,NaN,10.000000,70.0,NaN,7.005484,12.005483,7.005484,NaN,7.005484
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.100000,...,NaN,1.200548,11.200548,15.0,0.964356,9.351561,0.052109,9.351561,0.990071,10.341632
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.200000,...,NaN,1.200548,4.200548,6.0,0.954881,7.345366,0.045915,7.345366,0.872379,8.217746
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.300000,...,NaN,1.200548,3.200548,5.0,0.953539,7.344146,0.044694,7.344146,0.849194,8.193340
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,1.400000,...,NaN,1.200548,2.200548,4.0,0.954972,7.337388,0.037937,7.337388,0.720799,8.058187
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,1.500000,...,NaN,1.200548,5.200548,9.0,0.952623,9.367022,0.067570,9.367022,1.283828,10.650849
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,1.600000,...,NaN,1.200548,4.200548,8.0,0.950465,9.367331,0.067879,9.367331,1.289704,10.657035
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,1.700000,...,NaN,1.200548,3.200548,7.0,0.950049,9.362316,0.062864,9.362316,1.194420,10.556736
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,1.800000,...,NaN,1.200548,6.200548,13.0,0.956804,12.379458,0.080007,12.379458,1.520125,13.899584
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,1.900000,...,NaN,1.200548,5.200548,12.0,0.954343,12.381506,0.082055,12.381506,1.559036,13.940542


In [12]:
print(f"Length: {len(rdc_f_u_probability_array)}, p(0) = {rdc_f_u_probability_array[0]}")

Length: 21, p(0) = 1.7287087666744533e-06


In [13]:
rdc_f_u_probability_array

array([1.72870877e-06, 1.57434981e-05, 1.13569466e-04, 6.49041796e-04,
       2.93900365e-03, 1.05463995e-02, 2.99942869e-02, 6.76159001e-02,
       1.20829203e-01, 1.71172804e-01, 1.92244476e-01, 1.71172804e-01,
       1.20829203e-01, 6.76159001e-02, 2.99942869e-02, 1.05463995e-02,
       2.93900365e-03, 6.49041796e-04, 1.13569466e-04, 1.57434981e-05,
       1.72870877e-06])